# 아이돌보미 관련 맘카페 게시글 감정분석

In [8]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats
from konlpy.tag import Okt
okt = Okt()
from mecab import MeCab
mecab = MeCab()
import pyLDAvis
import pyLDAvis.lda_model

# 데이터 로드 

## 감정 사전 로드

In [28]:
sentiword = pd.read_json('../../dataset/SentiWord_info.json')
sentiword

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


## 게시글 데이터 로드

In [26]:
baby_care_reaction = pd.read_json('../../dataset/baby_care_reaction.json')
baby_care_reaction

,_id,title,date,contents,like,review
0,{'$oid': '66af99c5bea1facc9e97637b'},아이돌보미 서비스 신청 결제 관련해서 여쭤봅니다,2024-08-04 12:29:00,"안녕하세요.\n여행 다녀와서 애기, 저, 친정엄마 세명 다 코로나 확진\n판정 받고...",0,NaN
1,{'$oid': '66af99cebea1facc9e97637c'},아이돌보미 문의,2024-08-04 11:38:00,7살아들~~내년에 초등학교입학입니다.\n입학하면 수업이 일찍 끝나서 그 이후 학원 ...,0,NaN
2,{'$oid': '66af99d0bea1facc9e97637d'},어린이집과 정부지원아이돌보미는 같이 이용 못하는건가요?,2024-08-02 21:19:00,아예 처음 이용해보려고 알아보고 있어서요! 두개 중복은 안되는건가요? 어린이집을 오...,0,저이번에 신청했는데\n아이 하원후 시간 적용되서\n그렇게 한다고 했어요
3,{'$oid': '66af99d0bea1facc9e97637e'},어린이집과 정부지원아이돌보미는 같이 이용 못하는건가요?,2024-08-02 21:19:00,아예 처음 이용해보려고 알아보고 있어서요! 두개 중복은 안되는건가요? 어린이집을 오...,0,하원후라면 만약 1시에 제가 하원해도 이용가능한건가요? 아니면 어린이집 정규하원시간...
4,{'$oid': '66af99d0bea1facc9e97637f'},어린이집과 정부지원아이돌보미는 같이 이용 못하는건가요?,2024-08-02 21:19:00,아예 처음 이용해보려고 알아보고 있어서요! 두개 중복은 안되는건가요? 어린이집을 오...,0,올바로우\n그등원시간에 선생님 부르면 안된다\n이뜻 같았어요 그래서 중복불가\n애가...
...,...,...,...,...,...,...
26068,{'$oid': '66afd16abea1facc9e97c94f'},정부지원 아이돌보미 서비스...황당해요,2017-10-25 17:39:00,"★ 잠깐! 게시글 작성 전, 필독 공지! ★\n- 카페규정 : http://cafe...",2,그런분이 일하시면 맘님만 스트레스일듯요 다른이모님으로 바꾸시고 복직예정이면 홈cct...
26069,{'$oid': '66afd16abea1facc9e97c950'},정부지원 아이돌보미 서비스...황당해요,2017-10-25 17:39:00,"★ 잠깐! 게시글 작성 전, 필독 공지! ★\n- 카페규정 : http://cafe...",2,정식으로 일하는건 3월이요 얼집보내기로 했어요....\ncctv 고민이네요 ㅠㅠ
26070,{'$oid': '66afd16abea1facc9e97c951'},정부지원 아이돌보미 서비스...황당해요,2017-10-25 17:39:00,"★ 잠깐! 게시글 작성 전, 필독 공지! ★\n- 카페규정 : http://cafe...",2,아이돌봄 2년동안 3명정도 도우미와 지내봤는데~ 그런분 없었어요. 그 도우미는 아마...
26071,{'$oid': '66afd16abea1facc9e97c952'},정부지원 아이돌보미 서비스...황당해요,2017-10-25 17:39:00,"★ 잠깐! 게시글 작성 전, 필독 공지! ★\n- 카페규정 : http://cafe...",2,"저도 전혀 cctv 없어도 밑고 맡길만큼 잘해주시는데, 애기 젖병이랑 얼집갈 준비도..."


# 데이터 전처리

## 필요없는 데이터 삭제

In [31]:
baby_care_reaction = baby_care_reaction.groupby(['title','date','contents','like']).review.count().reset_index()

,title,date,contents,like,review
0,(+) 아동학대 관련은 신상 다 까발려야하지 않나요?,2020-09-15 01:07:00,산후도우미\n산부인과\n산후조리원\n아이돌보미\n어린이집\n\n요즘 인격 고장난 인...,4,24
1,(구인)아이돌보미 활동 자격되시는분,2019-03-25 20:47:00,오늘 면접보고 낼모레부터 출근하게되서요 \n개인아니구요.\n정부지원 되는 가정이고 ...,0,0
2,(다자녀)정부지원 아이돌보미 이용하시는분들,2023-08-09 00:13:00,서울 광진구 거주중인데 27개월 첫째 신청햇더니 보통6개월 대기라고합니다. 다른지역...,1,10
3,(동탄) 화성시아이돌보미 서비스 신청하면 선생님 금방 연결되나요?,2018-05-06 00:13:00,6월이면 아기가 100일이 되어서 화성시아이돌보미 서비스 이용해보려고해요. 정부지...,1,4
4,"(마감)아이돌보미 , 아기돌보미 지원합니다.",2019-10-14 00:32:00,보육교사를 하고 이번에 고시시험을 준비하면서 개인적인 시간이 필요하여\n개인 아기돌...,0,15
...,...,...,...,...,...
2039,휴원과 원격수업 시 아이돌봄서비스의 정부지원 비율이 확대되었습니다.^^,2020-11-26 14:56:00,안녕하세요. 지난번 아이돌봄서비스 이용 절차에 관해 글을 올린 적이 있었는데요.\n...,0,0
2040,휴직을쓸까요 시댁에 들어갈까요?,2022-01-23 22:47:00,안녕하세요. 6살된 공주키우슨 맞벌이 맘 이에요.\n원래 등원 친정엄마가 해주시다가...,0,7
2041,휴직중에 정부지원도우미 가능하나요?,2020-11-16 13:23:00,육아휴직기간동안에 아이돌보미 가능한가요?\n종일제 시간제 둘다 안되나요??,0,0
2042,힘드네요,2018-09-21 14:26:00,처음에 아이데리고 집갔을때도 일할때 아이돌보미 쓴다고했는데 그냥 돈 100만원 주라...,2,5
